In [ ]:
#ЗАДАЧА на будущее: автоматизировать подключение к выгрузкам
#к БД или хотя бы к csv файлам.

#ИНТРО
#https://youtu.be/-zps6hm0nX8?t=2450
#На видео Анатолий Карпов приводит методологию и пример

#У нас две маленьких выборки.
#(Лучше вообще не сравнивать).
#Но если надо, используем валидацию t-testa, Mann-Whitneyu, бутстрапа.
#Многократно проводим на одной из выборок A/A тесты.
#Смотрим, какой метод работает "лучше" (показывает отсутствие различий).
#Этот способ должен быть самым крайним. Делать выводы таким образом опасно.
#Но все это лучше, чем на глазок.

import numpy as np
from scipy import stats

#Валидация методов через A/A тест
#Валидируем t-test и Mann-Whitneyu
class Validation():
  def __init__(self, a, trials=100, alpha = 0.05):
    self.a = a
    self.trials = trials
    self.alpha = alpha
  
  def pre_test(self):
    self.method_validation(stats.mannwhitneyu)
    self.method_validation(stats.ttest_ind)

    permutation = Permutation(self.trials, self.alpha)  
    self.method_validation(permutation.mean)
    self.method_validation(permutation.median)

  #A/A тест для валидации метода
  def method_validation(self, method):
    #Тестируем бутстрап
    if (method.__name__ == 'mean') or (method.__name__ == 'median'):
      #Первое А
      a = np.random.choice(self.a, len(self.a))
      #Второе А
      b = np.random.choice(self.a, len(self.a))
      inference = method(a, b)
      p_value = inference[1]
      print(p_value)
      results = p_value
      p = self.p_test(results, method)

    #Тестируем все остальное (t-test и Mann-Whitneyu)
    else:
      results = []
      trial = 0
      while (trial < self.trials):
        a = np.random.choice(self.a, len(self.a))
        b = np.random.choice(self.a, len(self.a))
        if ( list(a) == list(b) ):
          continue
        else:
          inference = method(a, b)
          p_value = inference[1]
          if p_value >= self.alpha:
            results.append(0)
          else:
            results.append(1)
          trial += 1
      p = self.p_test(results, method)

  def p_test(self, results, method):
    if (isinstance(results, float)):
      p = results
      p = p * 100
      p = round(p, 2)
      if p == 0:
      #ноль может быть при очень малом количестве испытаний, поэтому меняем его дефолтное значение
        p = 0.0001
        print(f'{method.__name__}: False positive rate is %{p}')
        return p

      #А/A тест показывает различия - это плохо, метод не валидный
      else:
        print(f'{method.__name__}: False positive rate is too high - %{p}')
        print(f'{method}: is not recommended for use' )
        return p
    
    else:
      p = results.count(1)/len(results) * 100
      p = round(p, 2)
      print(p)
      if p == 0:
        #ноль может быть при очень малом количестве испытаний, поэтому меняем его дефолтное значение
        p = 0.0001
        print(f'{method}: False positive rate is %{p}')
        return p

      #А/A тест показывает различия - это плохо, метод не валидный
      else:
        print(f'{method}: False positive rate is too high - %{p}')
        print(f'{method}: is not recommended for use' )
        return p

#Бутстрап - сравниваем  
class Permutation():
  def __init__(self, trials=100, alpha = 0.05):
    self.trials = trials
    self.alpha = alpha

  def mean(self, a, b):
    mean_a = np.mean(a)
    mean_b = np.mean(b)
    #diff_observed = abs(mean_a - mean_b)
    diff_observed = abs(mean_a - mean_b)

    results = []
    trial = 0
    while (trial < self.trials):
      combination = a + b
      shuffled = combination
      np.random.shuffle(shuffled)
      a = shuffled[:len(a)]
      b = shuffled[-len(b):]

      mean_a = np.mean(a)
      mean_b = np.mean(b)
      #diff_permutated = abs(mean_a - mean_b)
      diff_permutated = mean_a - mean_b
      if (diff_permutated > diff_observed):
        results.append(1)
      else:
        results.append(0)
      trial += 1
    p = [self.alpha, results.count(1)/len(results)]
    #print(results.count(1))
    #print(trial)
    return p

  def median(self, a, b):
    median_a = np.median(a)
    median_b = np.median(b)
    diff_observed = median_a - median_b
    #diff_observed = abs(median_a - median_b)
    results = []
    trial = 0
    while (trial < self.trials):
      combination = a + b
      shuffled = combination
      np.random.shuffle(shuffled)
      a = shuffled[:len(a)]
      b = shuffled[-len(b):]

      median_a = np.median(a)
      median_b = np.median(b)
      diff_permutated = median_a - median_b
      #diff_permutated = abs(median_a - median_b)
      if (diff_permutated > diff_observed):
        results.append(1)
      else:
        results.append(0)
      trial += 1
    p = [self.alpha, results.count(1)/len(results)]
    #print(results.count(1))
    #print(trial)
    return p

In [ ]:
#Data 1 vs Data 2 
x = [24, 661, 707, 106, 71, 262]
y = [125, 225, 608, 402]
test = Validation(x, 10000, 0.05)
test.pre_test()

8.34
<function mannwhitneyu at 0x7f8d1abd9050>: False positive rate is too high - %8.34
<function mannwhitneyu at 0x7f8d1abd9050>: is not recommended for use
5.12
<function ttest_ind at 0x7f8d1abd1950>: False positive rate is too high - %5.12
<function ttest_ind at 0x7f8d1abd1950>: is not recommended for use
0.0
mean: False positive rate is %0.0001
0.0
median: False positive rate is %0.0001


In [ ]:
permutation = Permutation(10000, 0.05)

print(stats.mannwhitneyu(x, y))
print(stats.ttest_ind(x, y))
print(permutation.mean(x, y))
print(permutation.median(x, y))

MannwhitneyuResult(statistic=10.0, pvalue=0.37455956650029765)
Ttest_indResult(statistic=-0.19727328908358727, pvalue=0.8485336476483601)
[0.05, 0.4247]
[0.05, 0.6429]
